In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Electric_Vehicle_Population_Data.xlsx to Electric_Vehicle_Population_Data.xlsx


In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import spearmanr, kendalltau
from sklearn.cluster import KMeans

# Load Dataset (Already Uploaded by User)
df = pd.read_excel("Electric_Vehicle_Population_Data.xlsx")

# Extract necessary columns
columns_needed = ["County", "Make", "Model", "Electric Vehicle Type", "Electric Range", "Base MSRP", "Electric Utility"]
df = df[columns_needed]

# Aggregate data at the county level
county_data = df.groupby("County").agg(
    EV_Adoption=("Make", "count"),
    Charging_Providers=("Electric Utility", "nunique")
).dropna()

# Regression Analysis
# Prepare independent and dependent variables
X = sm.add_constant(county_data["Charging_Providers"])
y = county_data["EV_Adoption"]

# Fit linear regression model
model = sm.OLS(y, X).fit()

# Polynomial Regression
county_data["Charging_Providers^2"] = county_data["Charging_Providers"] ** 2
X_poly = sm.add_constant(county_data[["Charging_Providers", "Charging_Providers^2"]])
model_poly = sm.OLS(y, X_poly).fit()

# Logarithmic Regression
county_data["Log_Charging_Providers"] = np.log1p(county_data["Charging_Providers"])
X_log = sm.add_constant(county_data["Log_Charging_Providers"])
model_log = sm.OLS(y, X_log).fit()

# Correlation Analysis
spearman_corr, spearman_p = spearmanr(county_data["Charging_Providers"], county_data["EV_Adoption"])
kendall_corr, kendall_p = kendalltau(county_data["Charging_Providers"], county_data["EV_Adoption"])

# Clustering Analysis (K-Means)
kmeans = KMeans(n_clusters=3, random_state=42)
county_data["Cluster"] = kmeans.fit_predict(county_data[["Charging_Providers", "EV_Adoption"]])

# Print Regression Summaries and Correlation Results
print("\nLinear Regression Summary:\n", model.summary())
print("\nPolynomial Regression Summary:\n", model_poly.summary())
print("\nLogarithmic Regression Summary:\n", model_log.summary())
print(f"\nSpearman Correlation: {spearman_corr}, P-Value: {spearman_p}")
print(f"Kendall Correlation: {kendall_corr}, P-Value: {kendall_p}")
print("\nK-Means Cluster Centers:\n", kmeans.cluster_centers_)

# Visualizations
# Figure 1: Scatter Plot with Regression Line
plt.figure(figsize=(7, 5))
sns.regplot(x=county_data["Charging_Providers"], y=county_data["EV_Adoption"],
            scatter_kws={"s": 70, "alpha": 0.7}, line_kws={"color": "red"})
plt.xlabel("Number of Charging Providers")
plt.ylabel("EV Adoption Count")
plt.title("Figure 1: Relationship Between Charging Infrastructure and EV Adoption")
plt.grid(True)
plt.show()

# Figure 2: Bar Chart of Top 10 Counties by EV Adoption
plt.figure(figsize=(7, 5))
top_counties = county_data.sort_values("EV_Adoption", ascending=False).head(10)
sns.barplot(x=top_counties["EV_Adoption"], y=top_counties.index, palette="Blues_r")
plt.xlabel("Total EV Adoption")
plt.ylabel("County")
plt.title("Figure 2: Top 10 Counties by EV Adoption")
plt.grid(axis="x")
plt.show()

# Figure 3: Line Chart - EV Growth vs. Infrastructure Expansion
plt.figure(figsize=(7, 5))
sorted_df = county_data.sort_values("Charging_Providers")
plt.plot(sorted_df["Charging_Providers"], sorted_df["EV_Adoption"], marker="o", linestyle="-", color="blue")
plt.xlabel("Number of Charging Providers")
plt.ylabel("EV Adoption Count")
plt.title("Figure 3: EV Adoption Growth as Charging Infrastructure Expands")
plt.grid(True)
plt.show()

# Figure 4: Clustered Scatter Plot (K-Means Clustering)
plt.figure(figsize=(7, 5))
sns.scatterplot(x=county_data["Charging_Providers"], y=county_data["EV_Adoption"], hue=county_data["Cluster"],
                palette="coolwarm", s=80)
plt.xlabel("Number of Charging Providers")
plt.ylabel("EV Adoption Count")
plt.title("Figure 4: County Segmentation Based on EV Adoption & Charging Infrastructure")
plt.grid(True)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Electric_Vehicle_Population_Data.xlsx'